In [1]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,"../")
import mplhep as hep
import pickle
import glob
import ROOT as rt
import coffea
import awkward as ak
from coffea import hist, processor
from coffea.nanoevents.methods import candidate
from coffea.nanoevents.methods import vector
import pandas as pd

Welcome to JupyROOT 6.28/00


In [2]:
sys.path.append("/uscms/home/amalbert/nobackup/CMSSW_14_1_0_pre4/src/RazorCommon/Tools/bin")
import importlib
import getMuonScaleFactor

In [3]:
#paths
MC_path = "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/MC_noClusters/MC_Summer23/DYto2Mu_MLL-50to120_Merged/DYto2Mu_MLL-50to120_Merged_18666pb_weighted.root"

data_path_list = ["/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023B_PromptReco-v1_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023B_PromptReco-v1_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023C_PromptReco-v1_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023C_PromptReco-v1_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023C_PromptReco-v2_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023C_PromptReco-v2_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023C_PromptReco-v3_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023C_PromptReco-v3_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023C_PromptReco-v4_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023C_PromptReco-v4_goodLumi.root"]

In [4]:
ak.behavior.update(candidate.behavior)

def getLZDF(f,nEvents=-1,version="new"): #lazy dataframe with events that have cluster matched to probe muon
    events_raw = uproot.open(f)['MuonSystem']
    df = coffea.processor.LazyDataFrame(events_raw,entrystop=nEvents)
    start,stop = df._branchargs['entry_start'],df._branchargs['entry_stop']
    events = uproot.lazy(df._tree)
    #events = events[start:stop]
    return events

In [5]:
#events_data = getLZDF(data_path)
events_MC_full = getLZDF("root://cmseos.fnal.gov/"+MC_path)

In [6]:
data_events = [getLZDF("root://cmseos.fnal.gov/"+data_path) for data_path in data_path_list]

In [7]:
events_data_full = ak.concatenate(data_events, axis=0)

#events_data_full = events_data_full[events_data_full.nCscRechitClusters>0]
events_data_full = events_data_full[np.logical_and(events_data_full.ZMass>50, events_data_full.ZMass<120)]

In [62]:
events_data_full["lepTag"]
lepTag_array = events_data_full["lepTag"]
lepTag_array = np.repeat(lepTag_array,np.array(events_data_full["numTag"]), axis=0)
arr = np.zeros(np.shape(lepTag_array)[0], dtype=bool)
arr[::2] = 1  # Set even indices to 1
mask = ak.all(lepTag_array, axis=1)
print(lepTag_array)
#print(mask)
mask_T_F = np.logical_and(arr, mask)
mask_F_T = np.logical_and(np.logical_not(arr), mask)

#indices_T_F = np.zip()
lepTag_array = np.array(lepTag_array)
lepTag_array.reshape((np.array(mask_T_F).shape[0],2))
#print(mask_T_F)
#print(mask_F_T)
lepTag_array[mask_T_F] = [True, False]
lepTag_array[mask_F_T] = [False, True]
print(lepTag_array)

[[False, True], [True, True], [True, True, ... True], [True, True], [True, True]]
[[False  True]
 [False  True]
 [ True False]
 ...
 [ True False]
 [False  True]
 [ True False]]


In [67]:
np.where(lepTag_array==True)[1]

array([1, 1, 0, ..., 0, 1, 0])

In [68]:
np.where(lepTag_array==False)[1]

array([0, 0, 1, ..., 1, 0, 1])

In [9]:
branch_names = ["weight", "pileupWeight", "ZMass", "met", "metPhi", "Flag_all", "Flag_ecalBadCalibFilter", "jetVeto"]

In [ ]:
def getBranches(LZDF, isMC=False):
    for branch in branch_names:
        if (not isMC) and (branch in ["weight", "pileupWeight"]):
            continue
        new_df = ak.with_field(new_df, np.repeat(LZDF[branch],LZDF["numTag"]), branch)
    print("finished event level variables")
    
    lepTag_array = LZDF["lepTag"]
    lepTag_array = np.repeat(lepTag_array,np.array(LZDF["numTag"]), axis=0)
    arr = np.zeros(np.shape(lepTag_array)[0], dtype=bool)
    arr[::2] = 1  # Set even indices to 1
    mask = ak.all(lepTag_array, axis=1)
    mask_T_F = np.logical_and(arr, mask)
    mask_F_T = np.logical_and(np.logical_not(arr), mask)
    lepTag_array = np.array(lepTag_array)
    lepTag_array.reshape((np.array(mask_T_F).shape[0],2))
    lepTag_array[mask_T_F] = [True, False]
    lepTag_array[mask_F_T] = [False, True]
    
    row_indices = np.arange(lepTag_array.shape[0], dtype=int)
    column_indices_probe =np.where(lepTag_array==False)[1]
    column_indices_tag =np.where(lepTag_array==True)[1]
    
    
    if isMC:
        MC_SF_LooseID = getMuonScaleFactor.getLooseIDEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_LooseISO = getMuonScaleFactor.getLooseISOEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_TightID = getMuonScaleFactor.getTightIDEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_TightISO = getMuonScaleFactor.getTightISOEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_HLT = getMuonScaleFactor.getHLTEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        
        MC_SF_LooseID = np.repeat(MC_SF_LooseID,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
        MC_SF_LooseISO = np.repeat(np.array(MC_SF_LooseISO),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
        MC_SF_TightID = np.repeat(MC_SF_TightID,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
        MC_SF_TightISO = np.repeat(MC_SF_TightISO,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
        MC_SF_HLT = np.repeat(MC_SF_HLT,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
        
        MC_Weight_Total = new_df["weight"]*MC_SF_LooseID*MC_SF_LooseISO*MC_SF_TightID*MC_SF_TightISO*MC_SF_HLT
        new_df = ak.with_field(new_df, MC_Weight_Total, "weight_total")
    
    print("at muon variables")
    #load pT, eta, and phi for tag and probe muons
    probe_pT = np.repeat(np.array(LZDF["lepPt"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
    probe_eta = np.repeat(np.array(LZDF["lepEta"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
    probe_phi = np.repeat(np.array(LZDF["lepPhi"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
    
    tag_pT = np.repeat(np.array(LZDF["lepPt"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
    tag_eta = np.repeat(np.array(LZDF["lepEta"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
    tag_phi = np.repeat(np.array(LZDF["lepPhi"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
    
    new_df = ak.with_field(new_df, probe_pT, "probe_pT")
    new_df = ak.with_field(new_df, probe_eta, "probe_eta")
    new_df = ak.with_field(new_df, probe_phi, "probe_phi")
    
    new_df = ak.with_field(new_df, tag_pT, "tag_pT")
    new_df = ak.with_field(new_df, tag_eta, "tag_eta")
    new_df = ak.with_field(new_df, tag_phi, "tag_phi")
    
    return new_df